Black Scholes formula is valid for Europian option (can be exercied at the maturity date only):
    
1. The Black-Scholes model requires five input variables: the strike price of an option, the current stock price, the time to expiration, the risk-free rate, and the volatility. The higher the volatility is, the better!
2. Though usually accurate, the Black-Scholes model makes certain assumptions that can lead to predictions that deviate from the real-world results.

$$
    C = SN(d_1) - K \exp(-rt) N(d_2) \\ d_1 = \frac{ln(\frac{S}{K}) + (r + \frac{\sigma^2}{2}) t}{S \sqrt t} \\ 
    d_2 = \frac{ln(\frac{S}{K}) + (r -\frac{\sigma^2}{2}) t}{S \sqrt t}
$$

C: call option price
<br>
S: Current stock price
<br>
K: Strike price
<br>
r: risk-free interest rate
<br>
t: time to maturity
<br>
N: Cumulative normal distribution

In [1]:
import numpy as np
import pandas as pd
import yfinance as yfin
from scipy.stats import norm
from pandas_datareader import data as pdr

C:\Users\yaser\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def d1(S, K, r, stdev, T):
    return ( np.log(S/K) + (r + stdev**2 / 2) * T ) / (stdev * np.sqrt(T))
def d2(S, K, r, stdev, T):
    return ( np.log(S/K) + (r - stdev**2 / 2) * T ) / (stdev * np.sqrt(T))

In [3]:
def BSM(S,K,r,stdev,T):
    return (S * norm.cdf(d1(S,K,r,stdev,T))) - (K * np.exp(-r * T) * norm.cdf(d2(S, K, r, stdev, T)))

In [4]:
tickers = ['VTI']
yfin.pdr_override() #  you ensure compatibility with the Yahoo Finance API changes and possibly improve the reliability and functionality of data retrieval from Yahoo Finance within your code
data = pd.DataFrame()
for t in tickers:
    data[t] = pdr.get_data_yahoo(t, start="2003-3-30", end="2024-3-30")['Adj Close']

[*********************100%%**********************]  1 of 1 completed


In [5]:
# stock price in the last day
S = data.iloc[-1]
S

VTI    259.899994
Name: 2024-03-28 00:00:00, dtype: float64

In [6]:
log_returns = np.log(data/data.shift(1))
log_returns.head()

,VTI
Date,
2003-03-31,NaN
2003-04-01,0.014380
2003-04-02,0.023678
2003-04-03,-0.005837
2003-04-04,0.003530


In [7]:
log_returns[np.isnan(log_returns)] = 0
log_returns.head()

,VTI
Date,
2003-03-31,0.000000
2003-04-01,0.014380
2003-04-02,0.023678
2003-04-03,-0.005837
2003-04-04,0.003530


In [8]:
# annual standard deviation
stdev = log_returns.std() * 250 ** 0.5
stdev

VTI    0.189016
dtype: float64

In [9]:
r = 0.043
K = S * 1.1
T = 1

In [10]:
BSM(S, K, r, stdev, T)

VTI    13.877983
Name: 2024-03-28 00:00:00, dtype: float64